In [14]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("data/stream_e.csv", sep = "ef5r64i", engine = "python")
print(df.columns, df.shape)


Index(['Texto', 'IdUsuario', 'FechaCreacion', 'IdTweet', 'NombreUsuario',
       'FechaCreacionTweet', 'Hashtags', 'Url'],
      dtype='object') (1268, 8)


In [5]:
#Creating the dict with unique users and a list for the tweets they have written
corpus = {}
for i in range(df.shape[0]):
    if df.iloc[i]["NombreUsuario"] in corpus:
        corpus[df.iloc[i]["NombreUsuario"]].append(df.iloc[i]["Texto"])
    else:
        corpus[df.iloc[i]["NombreUsuario"]] = [df.iloc[i]["Texto"]]
type(corpus), len(corpus)

(dict, 1189)

In [70]:
documents = list(df.Texto)

In [73]:
#LSA

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline
  
# raw documents to tf-idf matrix: 
vectorizer = TfidfVectorizer(stop_words='english', 
                             use_idf=True, 
                             smooth_idf=True)
# SVD to reduce dimensionality: 
svd_model = TruncatedSVD(n_components=100,         # num dimensions
                         algorithm='randomized',
                         n_iter=10)
# pipeline of tf-idf + SVD, fit to and applied to documents:
svd_transformer = Pipeline([('tfidf', vectorizer), 
                            ('svd', svd_model)])
svd_matrix = svd_transformer.fit_transform(documents)

# svd_matrix can later be used to compare documents, compare words, or compare queries with documents

In [90]:
#LDA

from gensim.corpora.Dictionary import load_from_text, doc2bow
from gensim.corpora import MmCorpus
from gensim.models.ldamodel import LdaModel
document = "This is some document..."
# load id->word mapping (the dictionary)
id2word = load_from_text('wiki_en_wordids.txt')
# load corpus iterator
mm = MmCorpus('wiki_en_tfidf.mm')
# extract 100 LDA topics, updating once every 10,000
lda = LdaModel(corpus=mm, id2word=id2word, num_topics=100, update_every=1, chunksize=10000, passes=1)
# use LDA model: transform new doc to bag-of-words, then apply lda
doc_bow = doc2bow(document.split())
doc_lda = lda[doc_bow]
# doc_lda is vector of length num_topics representing weighted presence of each topic in the doc

ModuleNotFoundError: No module named 'gensim'